<a href="https://colab.research.google.com/github/abxda/COLMEX-ML/blob/main/Semana_05_01_COLMEX.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
"""
Script para la estratificación de datos de servicios en viviendas utilizando K-Means.
Se muestran dos estrategias:
    1) Uso de variables en valores absolutos con estandarización (StandardScaler) para K-Means.
       - K-Means es sensible a la escala de las variables, por lo que se emplea StandardScaler para igualar la varianza.
    2) Uso de proporciones (variables relativas) respecto al total de viviendas habitadas.
       - Al calcular proporciones, se elimina el sesgo de magnitud por el total de viviendas, lo que puede generar clusters
         que reflejen mejor la penetración de cada servicio.

Requiere: duckdb, geopandas, scikit-learn, pandas, shapely, numpy
"""

'\nScript para la estratificación de datos de servicios en viviendas utilizando K-Means.\nSe muestran dos estrategias:\n    1) Uso de variables en valores absolutos con estandarización (StandardScaler) para K-Means.\n       - K-Means es sensible a la escala de las variables, por lo que se emplea StandardScaler para igualar la varianza.\n    2) Uso de proporciones (variables relativas) respecto al total de viviendas habitadas.\n       - Al calcular proporciones, se elimina el sesgo de magnitud por el total de viviendas, lo que puede generar clusters\n         que reflejen mejor la penetración de cada servicio.\n\nRequiere: duckdb, geopandas, scikit-learn, pandas, shapely, numpy\n'

In [ ]:
pip install duckdb geopandas scikit-learn --quiet

In [ ]:
# 1. Importar librerías necesarias
import duckdb
import geopandas as gpd
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
import pandas as pd
from shapely import wkb
import numpy as np

In [ ]:
# 2. Conexión a la base de datos DuckDB y configuración espacial
con = duckdb.connect('datos_inegi.duckdb')
con.execute("INSTALL spatial;")
con.execute("LOAD spatial;")

In [ ]:
# 3. Extracción de datos desde la base de datos
query = """
SELECT
    VPH_CEL,    -- Total de Viviendas con Teléfono celular
    VPH_INTER,  -- Total de Viviendas con Internet
    VPH_STVP,   -- Total de Viviendas con TV de paga
    VPH_SPMVPI, -- Total de Viviendas con Servicios de streaming
    VPH_CVJ,    -- Total de Viviendas con Consolas de videojuegos
    TVIVHAB,    -- Total de Viviendas Habitadas
    ST_AsWKB(geometry) AS geometry  -- Geometría en formato WKB
FROM censo_geo;
"""
df = con.execute(query).fetchdf()



In [ ]:
# 4. Conversión a GeoDataFrame y carga de la geometría
gdf = gpd.GeoDataFrame(
    df.drop(columns="geometry"),  # Se evita duplicar la columna de geometría
    geometry=[wkb.loads(bytes(wkb_bin)) if wkb_bin is not None else None for wkb_bin in df["geometry"]],
    crs="EPSG:4326"
)

In [ ]:
#########################################
# PARTE A: CLUSTERING CON VALORES ABSOLUTOS
#########################################

# 5A. Selección de variables de interés y manejo de valores faltantes
variables = ['VPH_CEL', 'VPH_INTER', 'VPH_STVP', 'VPH_SPMVPI', 'VPH_CVJ']
X_abs = gdf[variables].fillna(0)

In [ ]:
# 6A. Estandarización de los datos
#    K-Means utiliza la distancia euclidiana, por lo que es esencial que cada variable contribuya de forma equitativa.
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_abs)

In [ ]:
# 7A. Aplicación de K-Means con 5 clusters
#    Se asigna a cada registro un 'estrato' según el cluster resultante.
kmeans_abs = KMeans(n_clusters=5, random_state=42)
gdf['estrato_abs'] = kmeans_abs.fit_predict(X_scaled)


In [ ]:
# 8A. Cálculo de medias por estrato para cada variable
medias_estratos_abs = gdf.groupby('estrato_abs')[variables].mean()


In [ ]:
# 9A. Asignación de colores según un score compuesto
#    Se estandarizan las medias de cada estrato y se suma para obtener un score que permita ordenar los estratos.
medias_norm_abs = (medias_estratos_abs - medias_estratos_abs.mean()) / medias_estratos_abs.std()
medias_estratos_abs['score'] = medias_norm_abs.sum(axis=1)
estratos_ordenados_abs = medias_estratos_abs.sort_values('score', ascending=False).index

#    Mapeo de colores: los estratos con mayor score se asignan colores 'Verde', intermedios 'Amarillo' y menores 'Rojo'.
mapeo_colores_abs = {
    estratos_ordenados_abs[0]: 'Verde',
    estratos_ordenados_abs[1]: 'Verde',
    estratos_ordenados_abs[2]: 'Amarillo',
    estratos_ordenados_abs[3]: 'Rojo',
    estratos_ordenados_abs[4]: 'Rojo'
}
gdf['color_abs'] = gdf['estrato_abs'].map(mapeo_colores_abs)

In [ ]:

# 10A. Exportar resultado a Shapefile y mostrar resultados
output_shp_abs = "estratificacion_valores_absolutos.shp"
gdf.to_file(output_shp_abs)
print("Resultados - Clusterización con valores absolutos:")
print("\nMedias por estrato:")
print(medias_estratos_abs.drop(columns='score'))
print("\nAsignación de colores:")
print(mapeo_colores_abs)

<ipython-input-14-d5ccd3516e9d>:3: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  gdf.to_file(output_shp_abs)


Resultados - Clusterización con valores absolutos:

Medias por estrato:
                 VPH_CEL    VPH_INTER    VPH_STVP  VPH_SPMVPI     VPH_CVJ
estrato_abs                                                              
0              20.260696    16.382842   10.057458    7.102067    4.662388
1             181.257699   156.381793  104.168478   88.448822   48.830616
2             395.596730   359.485014  253.359673  227.730245  125.741144
3              70.728492    58.104112   36.002552   26.260755   15.731684
4            1170.434783  1087.043478  794.695652  728.956522  394.478261

Asignación de colores:
{4: 'Verde', 2: 'Verde', 1: 'Amarillo', 3: 'Rojo', 0: 'Rojo'}


/usr/local/lib/python3.11/dist-packages/pyogrio/raw.py:723: RuntimeWarning: Normalized/laundered field name: 'estrato_abs' to 'estrato_ab'
  ogr_write(


In [ ]:
#########################################
# PARTE B: CLUSTERING CON PROPORCIONES
#########################################

# 11B. Calcular proporciones de cada servicio respecto al total de viviendas habitadas
#      Esto permite evaluar la penetración de cada servicio independientemente del tamaño absoluto de la vivienda.
gdf["CEL_PROP"]    = np.where(gdf["TVIVHAB"] == 0, 0, gdf["VPH_CEL"]    / gdf["TVIVHAB"])
gdf["INTER_PROP"]  = np.where(gdf["TVIVHAB"] == 0, 0, gdf["VPH_INTER"]  / gdf["TVIVHAB"])
gdf["STVP_PROP"]   = np.where(gdf["TVIVHAB"] == 0, 0, gdf["VPH_STVP"]   / gdf["TVIVHAB"])
gdf["SPMVPI_PROP"] = np.where(gdf["TVIVHAB"] == 0, 0, gdf["VPH_SPMVPI"] / gdf["TVIVHAB"])
gdf["CVJ_PROP"]    = np.where(gdf["TVIVHAB"] == 0, 0, gdf["VPH_CVJ"]    / gdf["TVIVHAB"])


In [ ]:
# 12B. Seleccionar las columnas de proporción para el clustering
vars_cluster_prop = ["CEL_PROP", "INTER_PROP", "STVP_PROP", "SPMVPI_PROP", "CVJ_PROP"]
X_prop = gdf[vars_cluster_prop].fillna(0)

In [ ]:
# 13B. Aplicación de K-Means (5 clusters) sobre las proporciones
#      Al usar proporciones, la escala ya es comparable, por lo que no es necesario estandarizar.
kmeans_prop = KMeans(n_clusters=5, random_state=42)
gdf["estrato_prop"] = kmeans_prop.fit_predict(X_prop)

In [ ]:
# 14B. Cálculo de medias por estrato (proporciones) y redondeo
medias_estratos_prop = gdf.groupby("estrato_prop")[vars_cluster_prop].mean().round(4)

In [ ]:
# 15B. Asignación de colores basada en un score compuesto
#      Se estandarizan las medias, se suman para formar un score y se ordenan los estratos.
medias_norm_prop = (medias_estratos_prop - medias_estratos_prop.mean()) / medias_estratos_prop.std()
medias_estratos_prop["score"] = medias_norm_prop.sum(axis=1)
estratos_ordenados_prop = medias_estratos_prop.sort_values("score", ascending=False).index

mapeo_colores_prop = {
    estratos_ordenados_prop[0]: 'Verde',
    estratos_ordenados_prop[1]: 'Verde',
    estratos_ordenados_prop[2]: 'Amarillo',
    estratos_ordenados_prop[3]: 'Rojo',
    estratos_ordenados_prop[4]: 'Rojo'
}
gdf["color_prop"] = gdf["estrato_prop"].map(mapeo_colores_prop)

In [ ]:
# 16B. Exportar el resultado de la clusterización basada en proporciones a Shapefile
output_shp_prop = "estratificacion_proporcion.shp"
gdf.to_file(output_shp_prop)
print(f"\nShapefile generado: {output_shp_prop}")

<ipython-input-20-be1d8bab3d1b>:3: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  gdf.to_file(output_shp_prop)
/usr/local/lib/python3.11/dist-packages/pyogrio/raw.py:723: RuntimeWarning: Normalized/laundered field name: 'estrato_abs' to 'estrato_ab'
  ogr_write(
/usr/local/lib/python3.11/dist-packages/pyogrio/raw.py:723: RuntimeWarning: Normalized/laundered field name: 'SPMVPI_PROP' to 'SPMVPI_PRO'
  ogr_write(
/usr/local/lib/python3.11/dist-packages/pyogrio/raw.py:723: RuntimeWarning: Normalized/laundered field name: 'estrato_prop' to 'estrato_pr'
  ogr_write(



Shapefile generado: estratificacion_proporcion.shp


In [ ]:
# 17B. Mostrar resultados de la clusterización basada en proporciones
print("\nResultados - Clusterización con proporciones:")
print("\nMedias por estrato (proporciones):")
print(medias_estratos_prop.drop(columns="score"))
print("\nScore por estrato:")
print(medias_estratos_prop[["score"]])
print("\nAsignación de colores:")
print(mapeo_colores_prop)

# 18. Cierre de la conexión a la base de datos
con.close()


Resultados - Clusterización con proporciones:

Medias por estrato (proporciones):
              CEL_PROP  INTER_PROP  STVP_PROP  SPMVPI_PROP  CVJ_PROP
estrato_prop                                                        
0               0.8986      0.6975     0.3830       0.2196    0.1608
1               0.0078      0.0106     0.0024       0.0008    0.0003
2               0.8751      0.5684     0.1842       0.0903    0.0719
3               0.9702      0.9332     0.7350       0.6558    0.3508
4               0.9350      0.8267     0.5501       0.3950    0.2439

Score por estrato:
                 score
estrato_prop          
0             0.449276
1            -6.945332
2            -1.791302
3             5.540234
4             2.747124

Asignación de colores:
{3: 'Verde', 4: 'Verde', 0: 'Amarillo', 2: 'Rojo', 1: 'Rojo'}


In [ ]:
# 19B. Mostrar las medias de los valores absolutos por estrato
# Utilizando la clusterización basada en proporciones (ejemplo B), calculamos las medias de las variables absolutas:
variables_abs = ['VPH_CEL', 'VPH_INTER', 'VPH_STVP', 'VPH_SPMVPI', 'VPH_CVJ']
medias_abs_por_estrato = gdf.groupby("estrato_prop")[variables_abs].mean().round(2)

print("\nMedias de valores absolutos por estrato (usando clusterización del ejemplo B):")
print(medias_abs_por_estrato)



Medias de valores absolutos por estrato (usando clusterización del ejemplo B):
              VPH_CEL  VPH_INTER  VPH_STVP  VPH_SPMVPI  VPH_CVJ
estrato_prop                                                   
0               42.68      32.64     17.52       11.42     8.66
1                0.08       0.07      0.03        0.02     0.00
2               26.61      17.65      7.97        4.38     3.53
3               49.10      47.04     36.38       33.43    17.92
4               43.38      37.89     24.98       18.75    12.09
